# SerialBox Tutorial

This notebook will walk through a simple example to get you familiar with using the basic features of SerialBox to save data from Fortran and load them into a Python script.

Below, the student will write their Fortran code below the "%%writefile serialBox_tutorial.F90" line.  This will let the student write out a Fortran file that he/she will be able to compile later.

In [12]:
%%writefile serialBox_tutorial.F90

program serialBox_tutorial

    implicit none

    integer          :: int0, ii, jj
    real             :: real0
    double precision :: dp0

    integer,          dimension(:), allocatable   :: int_arr0
    real,             dimension(:), allocatable   :: real_arr0
    double precision, dimension(:,:), allocatable :: dp_arr0

    !$ser init directory='./data' prefix='example' unique_id=.true.
    !$ser mode write
    !$ser on

    int0  = 7
    real0 = 7.7
    dp0   = 7.777

    allocate(int_arr0(7))
    allocate(real_arr0(7))
    allocate(dp_arr0(7,7))

    do ii = 1, int0
        int_arr0(ii) = ii
        real_arr0(ii) = ii + 0.1
        !write(*,*) 'int_arr0(', ii, ') = ', int_arr0(ii)
        !write(*,*) 'real_arr0(', ii, ') = ', real_arr0(ii)
    enddo

    do jj = 1, int0
        do ii = 1, int0
            dp_arr0(ii,jj) = (jj-1) + ii + 0.1
            !write(*,*) 'dp_arr0(', ii, ',', jj, ') = ', dp_arr0(ii,jj)
        enddo
    enddo

    !$ser savepoint 'input_data'
    !$ser data int0=int0 real0=real0 dp0=dp0 
    !$ser data int_arr0=int_arr0 real_arr0=real_arr0 dp_arr0=dp_arr0

    !$ser cleanup
end program

Overwriting serialBox_tutorial.F90


Once the student is able to write their Fortran file, they can run pp_ser.py Python script to write a Fortran file with the appropriate SerialBox calls in the Fortran code.  After that, their code is compiled using gfortran and executed.

In [13]:
%%bash
python3 ${SERIALBOX_ROOT}/python/pp_ser/pp_ser.py -s -v --output=s_serialBox_tutorial.F90 serialBox_tutorial.F90
gfortran -O3 -cpp -DSERIALIZE \
    -o tutorial_run s_serialBox_tutorial.F90 \
    -I${SERIALBOX_ROOT}/include \
    ${SERIALBOX_ROOT}/lib/libSerialboxFortran.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxC.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxCore.a \
    -lpthread -lstdc++ -lstdc++fs
rm -rf ./data
./tutorial_run

Processing file serialBox_tutorial.F90
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<
 >>> WARNING: SERIALIZATION IS ON <<<
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<


Once the student writes out their data, they can run the following python code to verify whether their data works or not.

In [11]:
#!/usr/bin/env python3

import numpy as np
import sys
import os
sys.path.append(os.environ.get('SERIALBOX_ROOT')+ '/python')
import serialbox as ser

serializer = ser.Serializer(ser.OpenModeKind.Read,'./data', 'example')

sp = serializer.get_savepoint('input_data')
#savepoints = serializer.savepoint_list()

#for sp in savepoints:
#print('Running savepoint ', sp[0])
int0  = serializer.read('int0',  sp[0])
real0 = serializer.read('real0', sp[0])
dp0   = serializer.read('dp0',   sp[0])

int_arr0  = serializer.read('int_arr0',  sp[0])
real_arr0 = serializer.read('real_arr0', sp[0])
dp_arr0   = serializer.read('dp_arr0',   sp[0])

int0_ref = 7
real0_ref = 7.7

int_arr0_ref = np.zeros(7)
real_arr0_ref = np.zeros(7, dtype='float32')
dp_arr0_ref = np.zeros((7,7))

for i in range(7):
    int_arr0_ref[i] = i + 1
    real_arr0_ref[i] = i + 1 + 0.1

for j in range(7):
    for i in range(7):
        dp_arr0_ref[i,j] = j + i+1 + 0.1

try:
    assert np.allclose(int0_ref, int0), "int0 does not match!"
    assert np.allclose(real0_ref, real0), "real0 does not match!"

    assert np.allclose(int_arr0_ref, int_arr0), "int_arr0 does not match!"
    assert np.allclose(real_arr0_ref, real_arr0), "real_arr0 does not match!"
    assert np.allclose(dp_arr0_ref, dp_arr0), "dp_arr0 does not match!"
except AssertionError as msg:
    print(msg)
    
print("Finished running comparison tests!")

Finished running comparison tests!


### Testing python code with serialized inputs and outputs
Now try an example with a little more complexity